<a href="https://colab.research.google.com/github/serinss/seSAC_ML_DL_Class/blob/main/05_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%run my_init.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%run my_init.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#정형 데이터와 비정형 데이터


#앙상블

통계역학에서, 어떤 계의 앙상블(ensemble)이란 그 계와 동등한 계의 모음을 말한다.

쉽게 말하면, 비슷한 무리들의 집합이라고 생각하시면 됩니다.

즉, 우리는 단일 모델에서 나오는 성능의 결과를 기대하는 것이 아니라,

여러 개의 단일 모델들의 평균치를 내거나, 투표를 해서 다수결에 의한 결정을 하는 등 여러 모델들의 집단 지성을 활용하여 더 나은 결과를 도출해 내는 것에 주 목적이 있습니다.

집단 지성을 활용하는 방법, 즉 앙상블 기법에는 다양한 방법이 있습니다.

- Voting (투표) - 투표를 통해 결과 도출
- Bagging - Bootstrap Aggregating (샘플을 다양하게 중복 생성)
- Boosting - 이전 오차를 보완하며 가중치 부여(일종의 오답노트)
- Stacking- 여러 모델을 기반으로 meta 모델

#앙상블 학습 - Voting, Bagging, RandomForest

---


https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

서로 다른 알고리즘이 도출해 낸 결과물에 대하여 최종 투표하는 방식을 통해 최종 결과를 선택

In [33]:
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier, \
  ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer

In [9]:
obj = load_breast_cancer()
type(obj)  # bunch 객체로 리턴 (딕셔너리와 비슷하다)

sklearn.utils.Bunch

In [10]:
print(f"feature_names: {obj.feature_names.shape} = Dimensionality\n{obj.feature_names}")
print("============================================================================\n")
print(f"data: {obj.data.shape} \n{obj.data}")
print("============================================================================\n")
print(f"target_names: {obj.target_names.shape}\n{obj.target_names}")
print("============================================================================\n")
print(f"target: {obj.target.shape}\n{obj.target}")

# 해당 데이터로 Voting해보기 위함임

feature_names: (30,) = Dimensionality
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']

data: (569, 30) 
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.8

In [11]:
X, y = load_breast_cancer(return_X_y=True)
print(f"X: {X.shape}\n{X}, \n\ny: {y.shape}\n{y}")

# X,y를 리턴해서 쓰면 데이터만 가져오는 것이므로 따로 뽑아서 쓰기 힘들다

X: (569, 30)
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]], 

y: (569,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train.shape, y_train.shape

((455, 30), (455,))

In [14]:
X_test.shape, y_test.shape

((114, 30), (114,))

In [31]:
lr = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

vt = VotingClassifier([('LR', lr), {'KNN', knn}]) # 디폴트 voting='hard'
bg = BaggingClassifier(dt, n_estimators=100) # 하나의 데이터로 부트스트래핑 샘플링을하여 다양한 모델을 학습
rf = RandomForestClassifier() # 기본적으로 dt로 고정됨

In [50]:
# for clf in [lr, knn, vt, dt, bg, rf]:
#   clf.fit(X_train, y_train)
#   y_pred = clf.predict(X_test)
#   print(f"{clf.__class__.__name__}: {accuracy_score(y_test, y_pred):.4f}")
  # 파이선 모든 객체는 __class__ 가 모두 존재함 -> 클래스에 대한 참조를 보면 이름이 존재

# Voting클래스가 제일 좋음 - 기본값으로 진행한 것임
# 일할 때, 앙상블을 안쓸래야 안쓸 수 없음 기본값으로만 진행해도 성능이 꽤 높아짐

### Hard Voting vs. Soft Voting
Hard Vote

classification을 예로 들어 보자면, 분류를 예측한 값이 1, 0, 0, 1, 1 이었다고 가정한다면 1이 3표, 0이 2표를 받았기 때문에 Hard Voting 방식에서는 1이 최종 값으로 예측을 하게 됩니다.

---------
Soft Vote

soft vote 방식은 각각의 확률의 평균 값을 계산한다음에 가장 확률이 높은 값으로 확정짓게 됩니다.

가령 class 0이 나올 확률이 (0.4, 0.9, 0.9, 0.4, 0.4)이었고, class 1이 나올 확률이 (0.6, 0.1, 0.1, 0.6, 0.6) 이었다면,

class 0이 나올 최종 확률은 (0.4+0.9+0.9+0.4+0.4) / 5 = 0.44, class 1이 나올 최종 확률은 (0.6+0.1+0.1+0.6+0.6) / 5 = 0.4 가 되기 때문에 앞선 Hard Vote의 결과와는 다른 결과 값이 최종 으로 선출되게 됩니다.

#랜덤 포레스트

In [17]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [18]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42) #최상의 성능(=빠른 수행속도)을 사용하기 위해 파라미터 값을 지정, n_jobs 
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #여기서 점수는 validation_score, train_score도 넣음
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과대적합 상태.. -> 이예제는 하이퍼파라미터 튜닝해도 결과가 잘 나아지지 않음(간단해서)

0.9973541965122431 0.8905151032797809


In [19]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

# 당도의 중요도가 감소하고 알코올,pH의 중요도가 상승함
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선태갛여 DT를 훈련하기 때문

[0.23167441 0.50039841 0.26792718]


In [20]:
rf.predict_proba(test_input)

array([[0.  , 1.  ],
       [0.85, 0.15],
       [0.02, 0.98],
       ...,
       [0.11, 0.89],
       [0.  , 1.  ],
       [0.01, 0.99]])

In [21]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) 
# out of bag (bagging샘플링 = 일부 -> oob = 취해지지 않은 데이터들)을 이용하여 부트스트랩 -> 교차 검증을 했을 때와 비슷한 결과가 나옴
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리

In [23]:
from pandas.core.common import random_state
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

In [25]:
scores['test_score']

array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665])

In [26]:
np.mean(scores['test_score']) # 동일 데이터를 전체를 가지고(전략 자체가 반대) 작업해도 점수가 크게 다르지 않음

0.8887848893166506

In [30]:
et.fit(train_input, train_target)
et.feature_importances_

# 랜덤 포레스트 : [0.23167441 0.50039841 0.26792718]
# 엑스트라 트리 : [0.20183568, 0.52242907, 0.27573525]
# 거의 동일

# 하지만, 랜덤의 경우 데이터가 작은 경우 결과가 잘 나오지 않음

array([0.20183568, 0.52242907, 0.27573525])

#그래디언트 부스팅
부스팅 -> 오답노트(약점 강화)  
그래디언트 -> 하나가 아닌 여러개를 취해서 샘플링(미니 배치같은)  
weak learner(약한 학습치, 예를 들어 DT)로 가중치를 더 줘가면서 학습 시킴  

In [35]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
gb.fit(train_input, train_target)

GradientBoostingClassifier(random_state=42)

In [38]:
accuracy_score(train_target, gb.predict(train_input))

0.8880123147969983

In [37]:
accuracy_score(test_target, gb.predict(test_input))

# 처음부터 weak learner로 시작하기 때문에 과대 적합일 일어나지 않음!
# 학습률을 높이고, 트리의 개수를 증가시키면 성능이 향상될 수 있음

0.8669230769230769

In [54]:
params = {
    'learning_rate': [0.05, 0.1, 0.5],
    'n_estimators': [500,1000, 5000],
    'subsample': [0.1, 0.2, 0.5, 1]
}

###CridSearchCV

In [55]:
gs = GridSearchCV(GradientBoostingClassifier(random_state=42), params, n_jobs=-1, cv=3)
gs.fit(train_input, train_target)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.5],
                         'n_estimators': [500, 1000, 5000],
                         'subsample': [0.1, 0.2, 0.5, 1]})

In [43]:
3*4*5 #개의 서로다른 모형들이 만들어 낸 학습 결과 중 best를 찾기

60

In [56]:
gs.best_score_

0.8797390420168739

In [57]:
gs.best_params_ # 듬성듬성 이동 - 학습률은 기존에 0.001, 0.05 수준이었음

{'learning_rate': 0.1, 'n_estimators': 500, 'subsample': 1}

In [58]:
gb = gs.best_estimator_
accuracy_score(test_target, gb.predict(test_input))

# validation 점수와 크게 다르지 않다.

0.8692307692307693

In [52]:
gb.feature_importances_ #fit 했으므로 볼 수 있음

array([0.1648851 , 0.66776294, 0.16735196])

#히스토그램 기반 그레이디언트 부스팅


In [65]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

In [66]:
scores["train_score"]

array([0.88717825, 0.88092374, 0.87854738, 0.88480038, 0.88239538])

In [67]:
scores["test_score"]

#굳

array([0.85673077, 0.87019231, 0.89220404, 0.86236766, 0.87295476])

In [68]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=42)
scores = cross_validate(lgbm, train_input, train_target, return_train_score=True)

In [69]:
scores['train_score']

array([0.93312485, 0.93432764, 0.93217893, 0.93241943, 0.93698894])

In [70]:
scores['test_score']

#약간의 오버피팅, MS에서 만들었는데 빠르고 최신 기술을 많이 적용하고 있어 인기가 높아지는 중

array([0.86442308, 0.88076923, 0.90856593, 0.86429259, 0.87680462])